In [7]:
import json
from copy import copy
import os

base_name = '_30cm'

# Load a COCO-style annotations file
with open(f'Guatemala{base_name}/annotations_coco.json', 'r') as f:
    gm_coco = json.load(f)

with open(f'SantoDomingo{base_name}/annotations_coco.json', 'r') as f:
    sd_coco = json.load(f)

with open(f'SanJose{base_name}/annotations_coco.json', 'r') as f:
    sj_coco = json.load(f)



In [8]:
merged_coco = dict()

sd_img_id = dict()
sj_img_id = dict()

img_ids = []
images = []
for i in gm_coco['images']:
    i = copy(i)
    i['file_name'] = f'Guatemala{base_name}/' + i['file_name'] 
    images.append(i)
    img_ids.append(i['id'])

max_id = max(img_ids) + 1
for i in sd_coco['images']:
    i = copy(i)
    i['file_name'] = f'SantoDomingo{base_name}/' + i['file_name'] 
    sd_img_id[i['id']] = i['id'] + max_id
    i['id'] = i['id'] + max_id
    images.append(i)
    img_ids.append(i['id'])  

max_id = max(img_ids) + 1
for i in sj_coco['images']:
    i = copy(i)
    i['file_name'] = f'SanJose{base_name}/' + i['file_name'] 
    sj_img_id[i['id']] = i['id'] + max_id
    i['id'] = i['id'] + max_id
    images.append(i)
    img_ids.append(i['id'])  

annotations = []
ann_ids = []
for i in gm_coco['annotations']:
    i = copy(i)
    annotations.append(i)
    ann_ids.append(i['id'])

max_id = max(ann_ids) + 1
for i in sd_coco['annotations']:
    i = copy(i)
    i['image_id'] = sd_img_id[i['image_id']]
    i['id'] = i['id'] + max_id
    annotations.append(i)
    ann_ids.append(i['id'])  

max_id = max(ann_ids) + 1
for i in sj_coco['annotations']:
    i = copy(i)
    i['image_id'] = sj_img_id[i['image_id']]
    i['id'] = i['id'] + max_id
    annotations.append(i)
    ann_ids.append(i['id'])  

categories = gm_coco['categories']

merged_coco['images'] = images 
merged_coco['annotations'] = annotations 
merged_coco['categories'] = categories 

In [9]:
from sklearn.model_selection import train_test_split as sklearn_train_test_split
train_ids, test_ids = sklearn_train_test_split(
    img_ids, test_size=0.2, random_state=42, shuffle=True
)

In [10]:
coco_train = dict()
coco_train['categories'] = categories 

images = []
for i in merged_coco['images']:
    if i['id'] in train_ids: 
        images.append(i)

annotations = []
for i in merged_coco['annotations']:
    if i['image_id'] in train_ids: 
        annotations.append(i)

coco_train['images'] = images 
coco_train['annotations'] = annotations        

In [11]:
coco_test = dict()
coco_test['categories'] = categories 

images = []
for i in merged_coco['images']:
    if i['id'] in test_ids: 
        images.append(i)

annotations = []
for i in merged_coco['annotations']:
    if i['image_id'] in test_ids: 
        annotations.append(i)

coco_test['images'] = images 
coco_test['annotations'] = annotations       

In [12]:
json_file = os.path.normpath(f"train_annotations_coco{base_name}.json")
with open(json_file, "w") as file:
    json.dump(coco_train, file, indent=4)  # `indent=4` for pretty formatting

json_file = os.path.normpath(f"test_annotations_coco{base_name}.json")
with open(json_file, "w") as file:
    json.dump(coco_test, file, indent=4)  # `indent=4` for pretty formatting